In [36]:
from openai import OpenAI

client = OpenAI()

tools = [{
    "type": "function",
    "function": {
        "name": "search_tavily",
        "description": "Search the web for a given query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "query"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "인터넷 검색 가능해?"}],
    tools=tools
)

print(completion.choices[0].message.tool_calls)

None


In [37]:
completion.choices[0].message.content

'응, 인터넷으로 검색할 수 있어. 도움이 필요하면 검색할 내용을 알려줘!'

In [11]:
import os
from tavily import TavilyClient
import json

In [12]:
tavily_client = TavilyClient(api_key=f"{os.getenv('TAVILY_API_KEY')}")

def search_tavily(query):
    response = tavily_client.search(
        query=query
    )
    return response['results']

In [30]:
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = search_tavily(args["query"])

In [31]:
result

[{'title': '9 Things That Are NEW in Paris for 2025! - france-hotel-guide.com',
  'url': 'https://www.france-hotel-guide.com/en/blog/new-in-paris-2025/',
  'content': 'Paris is always changing, and 2025 is bringing some incredible new attractions, events, and openings to the city. Why It’s Special: This ephemeral artwork underscores Paris’s commitment to contemporary art while celebrating its historical landmarks. After years of meticulous renovation, the iconic Grand Palais is set to reopen in 2025, returning as one of Paris’s most important cultural landmarks. Whether attending a high-profile event or simply admiring its grandeur, the reopening marks a new chapter for Paris’s artistic community. The Palais de la Découverte, Paris’s beloved science museum, will reopen in 2025 after a significant overhaul. Une publication partagée par Restaurant Le Mouton Blanc 🐑 (@lemoutonblanc_paris) Benihana Teppanyaki Restaurant Opening in Paris Une publication partagée par | Benihana • Paris | (@b

In [32]:
messages=[{"role": "user", "content": "What are landmarks in Paris 2025?"}]

In [33]:
messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

completion_2 = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [34]:
completion_2.choices[0].message.content

'In 2025, Paris will feature several new and rejuvenated landmarks alongside its traditional attractions:\n\n1. **Grand Palais**: After years of renovation, this iconic cultural landmark will reopen in 2025. It is set to resume its role as an important venue for events and exhibitions, marking a new chapter in Paris\'s artistic community.\n\n2. **Palais de la Découverte**: This beloved science museum will also reopen after a significant overhaul, offering visitors new scientific exhibits and experiences.\n\n3. **Seine-side Beaches**: New urban beaches will be introduced along the Seine, complementing the annual Paris Plages event. These new areas will allow locals and tourists to enjoy swimming and leisure activities by the river in central Paris.\n\n4. **Pont Neuf Transformation**: To celebrate its 40th anniversary, renowned artist JR, in collaboration with Fondation Christo et Jeanne-Claude, plans to transform the Pont Neuf into a grotto, offering a unique artistic experience.\n\n5. 